# active space test

In [13]:
import pennylane as qml
from pennylane import numpy as np

bohr2ang = 0.52917721092

# Define the molecule
symbols  = ['H', 'H', 'H']
geometry = np.array([[0.01076341,  0.04449877,  0.0],
                     [0.98729513,  1.63059094,  0.0],
                     [1.87262415, -0.00815842,  0.0]], requires_grad = False)/ bohr2ang
electrons = 3
charge = 0

# Build the electronic Hamiltonian
H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, charge=charge, mult=2, method="pyscf", 
                                            active_electrons=1, active_orbitals=2, basis="sto-3g")  # active_orbitals are spatial orbitals
# デフォルトのmethod="dhf"は変な解に収束してしまうので使わない
print(qubits)
print(H)
# Define the HF state
hf_state = qml.qchem.hf_state(1, 4)  # change nele and norbs into the ones within active space # Here, orbitals means spin orbitals (= qubits) 
print(hf_state)
# Generate single and double excitations
singles, doubles = qml.qchem.excitations(electrons, qubits) 
print(singles)
print(doubles)
# Map excitations to the wires the UCCSD circuit will act on
s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
print(s_wires)
print(d_wires)
# Define the device
dev = qml.device("default.qubit", wires=qubits)

# Define the qnode
@qml.qnode(dev)
def circuit(params, wires, s_wires, d_wires, hf_state):
    qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
    return qml.expval(H)

# Define the initial values of the circuit parameters
params = np.zeros(len(singles) + len(doubles))

# Define the optimizer
optimizer = qml.SPSAOptimizer(maxiter=100)
optimizer = qml.GradientDescentOptimizer(stepsize=0.3)

# Optimize the circuit parameters and compute the energy
for n in range(121):
    params, energy = optimizer.step_and_cost(circuit, params,
    wires=range(qubits), s_wires=s_wires, d_wires=d_wires, hf_state=hf_state)
    if n % 10 == 0:
        print("step = {:},  E = {:.8f} Ha, params[0] = {:.8f}".format(n, energy, params[0]))
print(params)

4
  (-0.826837067101442) [I0]
+ (-0.19788711575005355) [Z2]
+ (-0.19788711575005355) [Z3]
+ (-0.14105987074021184) [Z0]
+ (-0.14105987074021184) [Z1]
+ (-7.927662924994e-08) [Y1 Y3]
+ (-7.927662924994e-08) [X1 X3]
+ (1.4432347513702046e-07) [Y0 Y2]
+ (1.4432347513702046e-07) [X0 X2]
+ (0.06808935724853563) [Z0 Z2]
+ (0.06808935724853563) [Z1 Z3]
+ (0.07385775378272749) [Z0 Z3]
+ (0.07385775378272749) [Z1 Z2]
+ (0.13654139361855194) [Z2 Z3]
+ (0.18151044237875263) [Z0 Z1]
+ (1.9819157310967117e-08) [X0 Z1 X2]
+ (1.9819157310967117e-08) [Y0 Z1 Y2]
+ (1.9819157314002883e-08) [X1 Z2 X3]
+ (1.9819157314002883e-08) [Y1 Z2 Y3]
+ (-0.0057683965341918724) [Y0 Y1 X2 X3]
+ (-0.0057683965341918724) [X0 X1 Y2 Y3]
+ (-7.927662924994e-08) [Y0 Z1 Y2 Z3]
+ (-7.927662924994e-08) [X0 Z1 X2 Z3]
+ (1.4432347513702046e-07) [Z0 Y1 Z2 Y3]
+ (1.4432347513702046e-07) [Z0 X1 Z2 X3]
+ (0.0057683965341918724) [Y0 X1 X2 Y3]
+ (0.0057683965341918724) [X0 Y1 Y2 X3]
[1 0 0 0]
[[1, 3]]
[]
[[1, 2, 3]]
[]
step = 0,  E = 

In [ ]:
# 定義上パラメータは自分の実装の2倍になる